In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
import exifread
import glob, os
import shutil
from sklearn import decomposition
import sklearn 
import time

In [14]:
files = glob.glob("/Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-1_20141226/Negatives/*.JPG")
numneg=len(files)
print numneg
files = files+glob.glob("/Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-1_20141226/Positives/*.JPG")
num=len(files)
print num
filepos = glob.glob("/Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-1_20141226/Positives/*.JPG")
numpos=len(filepos)
print files[-1*numpos:-1][0]
direc='/Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-1_20141226/PCA/'
lenad=len('/Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-1_20141226/')


7224
7316
/Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-1_20141226/Positives/SM-1_20141205 (6801).JPG


In [15]:
dates=np.array([])
lat=np.array([])
lon=np.array([])
flash=np.array([])
lightsource=np.array([])
colorspace=np.array([])
# Open image file for reading (binary mode)
for i in range(num): 
    f = open(files[i], 'rb')
    tags = exifread.process_file(f)
    dates=np.append(dates,str(tags['EXIF DateTimeOriginal']).split()[0])
    lat= np.append(lat,str(tags['GPS GPSLatitude']))
    lon= np.append(lon,str(tags['GPS GPSLongitude']))
    flash=np.append(flash,str(tags['EXIF Flash']))
    lightsource=np.append(lightsource,str(tags['EXIF LightSource']))
    colorspace=np.append(colorspace,str(tags['EXIF ColorSpace']))
    
datepos=np.array([])
latpos=np.array([])
lonpos=np.array([])
flashpos=np.array([])
lightsourcepos=np.array([])
colorspacepos=np.array([])
# Open image file for reading (binary mode)
for i in range(numpos): 
    f = open(filepos[i], 'rb')
    tagpos = exifread.process_file(f)
    datepos=np.append(datepos,str(tagpos['EXIF DateTimeOriginal']).split()[0])
    latpos= np.append(latpos,str(tagpos['GPS GPSLatitude']))
    lonpos= np.append(lonpos,str(tagpos['GPS GPSLongitude']))
    flashpos=np.append(flashpos,str(tagpos['EXIF Flash']))
    lightsourcepos=np.append(lightsourcepos,str(tagpos['EXIF LightSource']))
    colorspacepos=np.append(colorspacepos,str(tagpos['EXIF ColorSpace']))

In [16]:
uni_lightsource =np.unique(lightsource)
uni_colorspace =np.unique(colorspace)
uni_flash =np.unique(flash)
uni_lon =np.unique(lon)
uni_lat =np.unique(lat)

uni_lightsourcepos =np.unique(lightsourcepos)
uni_colorspacepos =np.unique(colorspacepos)
uni_flashpos =np.unique(flashpos)
uni_lonpos =np.unique(lonpos)
uni_latpos =np.unique(latpos)
uni_latpos

array(['[37, 31, 35]'], 
      dtype='|S32')

In [ ]:
uniquedate =np.unique(dates)
print uniquedate
num_unique=np.zeros(len(uniquedate))
for i in range(len(uniquedate)):
    num_unique[i]=len(np.where(dates== uniquedate[i])[0])
num_unique

uniquedatepos =np.unique(datepos)
num_uniquepos=np.zeros(len(uniquedatepos))
for i in range(len(uniquedatepos)):
    num_uniquepos[i]=len(np.where(datepos== uniquedatepos[i])[0])
num_uniquepos

['2014:11:12' '2014:11:14' '2014:11:15' '2014:11:16' '2014:11:17'
 '2014:11:18' '2014:11:20' '2014:11:21' '2014:11:22' '2014:11:23'
 '2014:11:24' '2014:11:25' '2014:11:26' '2014:11:27' '2014:11:28'
 '2014:11:30' '2014:12:01' '2014:12:05' '2014:12:06' '2014:12:11'
 '2014:12:13' '2014:12:14' '2014:12:15' '2014:12:16' '2014:12:26']


array([  1.,   3.,   6.,   3.,   9.,  14.,   3.,  24.,  12.,   1.,   3.,
         6.,   7.])

In [ ]:
start_time = time.time()
for index in range(len(uniquedate)): 
    print 'Iteration over unique date of ',uniquedate[index] 
    id0=np.where(dates== uniquedate[index])
    print dates[id0][0]
    name=direc+dates[id0][0]
    if os.path.isdir(name):
        shutil.rmtree(name)
    if os.path.isdir(direc)== False:
        os.mkdir(direc)    
    os.mkdir(name)
    sx=1840
    sy=2592
    dx=sx/5
    dy=sy/5
    imcube=np.empty([368, 518])
    #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    for i in range(len(id0[0])):
        im=cv2.imread(files[id0[0][i]])
        im[1744:1944,0:209,:]=0
        im=im[0:1840,:,:]
        #im[1840:1944,:,:]=0
        img=cv2.resize(im, (dy, dx),interpolation=cv2.INTER_NEAREST)
        imresized=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        #imresized = clahe.apply(imresized)
        imcube=np.dstack((imcube,imresized))
    imcube=np.delete(imcube,0,2)

    imblur=np.empty([368,518])
    for i in range(len(id0[0])):
        imblur=np.dstack((imblur,cv2.blur(imcube[:,:,i],(5,5))))
    imblur=np.delete(imblur,0,2)

    #Calculating the mean blur image
    nx,ny,nz=np.shape(imblur)
    blurmean=np.mean(imblur,axis=2)
    imblur_center=np.zeros([nx*ny,nz])
    np.shape(imblur)
    for i in range(len(id0[0])):
        imblur_center[:,i]=(imblur[:,:,i]-blurmean).flatten()

    n_components=10
    estimator=decomposition.PCA(n_components=n_components, svd_solver='randomized',whiten=True)
    estimator.fit(imblur_center.T)

    blur_transformed=estimator.fit_transform(imblur_center.T)
    blur_inverse=estimator.inverse_transform(blur_transformed)

    ### calculating the similarity between the PCA blured inverse transformed image and the original image.
    stdpcablur=[]
    medianpcablur=[]
    for i in range(len(id0[0])):
        invblur=blur_inverse[i,:].reshape(368, 518)
        stdpcablur=np.append(stdpcablur,np.std(imblur_center[:,i].reshape(368, 518)-invblur))
        medianpcablur=np.append(medianpcablur,np.mean(imblur_center[:,i].reshape(368, 518)-invblur))

    # Plotting the distribution of stddev of the residual maps
    minstdblur=np.min(stdpcablur)
    maxstdblur=np.max(stdpcablur)

    n, bins, patches=plt.hist(stdpcablur/maxstdblur,bins=np.linspace(minstdblur/maxstdblur,1,70),facecolor='red', hatch='///',alpha=1.0,label=' Lon:'+lon[id0][0][1:-1]+'\n Lat:'+lat[id0][0][1:-1]+'\n Daylight')

    minmedianblur=np.min(medianpcablur)
    medianmaxblur=np.max(medianpcablur)

    stdpcablur_med=np.median(stdpcablur/maxstdblur)
    stdpcablur_16=np.percentile(stdpcablur/maxstdblur,16)
    sigma=stdpcablur_med-stdpcablur_16
    newthr=stdpcablur_med+3*sigma

    plt.axis([0.13, 1, 0, np.max(n)+5])
    #plt.grid(True)
    plt.xlabel('Normalized PCA Dissimilarity Score')
    plt.ylabel('Number of images')
    plt.title('Date: '+dates[id0][0])
    plt.plot([newthr,newthr],[0,np.max(n)+5],linewidth=2,color='blue',label=r'3\sigma')

    right=np.ones(np.max(n)+5)-0.5
    plt.hist(right,bins=np.linspace(newthr,1,2),facecolor='green', alpha=0.4)
    left=np.ones(np.max(n)+5)-0.8
    plt.hist(left,bins=np.linspace(0,newthr,2),facecolor='cyan', alpha=0.4)
    plotname=name+'/'+'PCA-Dissimilarity-'+name[-10:]+'.png'
    plt.savefig(plotname)

    # Saving the PCA residual maps
    for i in range(len(out3sigma[0])):
        newname=name+'/'+imlistblur[out3sigma[0][i]][lenad:lenad+1]+'_'+imlistblur[out3sigma[0][i]][lenad+10:-4]
        invblur=blur_inverse[i,:].reshape(368, 518)
        #cv2.imwrite(newname+'_PCA_InvTransformed.JPG',invblur+blurmean)
        #cv2.imwrite(newname+'_PCA_MeanSubOrg.JPG',imblur_center[:,i].reshape(368, 518)+blurmean)
        imres=imblur_center[:,i].reshape(368, 518)-invblur
        imres=imres+np.fabs(np.min(imres))
        imres=255*imres/np.max(imres)
        cv2.imwrite(newname+'_PCA_Residual.JPG',imres)

    # Saving the list of outlier candidates
    out3sigma=np.where(stdpcablur/maxstdblur >= newthr)
    newname=name+'/'+'Outliers-'+name[-10:]+'.txt'
    cand=open(newname,'w')
    for i in imlistblur[out3sigma[0]]:
        cand.write(i+'\n')
    cand.close()
print("--- %s seconds ---" % (time.time() - start_time))

